In [21]:
## google drive to google colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('/content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv')

Since there are many measures in the Measures coulumn after the EDA, we have goruped them into 2 different lists

In [24]:
prevention_cols = [
    'ACCESS2', 'BPMED', 'CHECKUP', 'CHOLSCREEN',
    'COLON_SCREEN', 'COREM', 'COREW', 'DENTAL', 'MAMMOUSE', 'PAPTEST']

behavior_cols = ['BINGE', 'CSMOKING', 'LPA', 'OBESITY', 'SLEEP']

outcome_cols = [
    'ARTHRITIS', 'BPHIGH', 'CANCER',
    'CASTHMA', 'CHD', 'COPD',
    'DIABETES', 'HIGHCHOL', 'KIDNEY',
    'MHLTH', 'PHLTH', 'STROKE', 'TEETHLOST']

columns_to_drop = [ 'Year', 'StateAbbr', 'DataSource', 'Measure','Data_Value_Unit',
                    'Data_Value_Footnote', 'Data_Value_Type', 'Low_Confidence_Limit',
                    'High_Confidence_Limit', 'Data_Value_Footnote_Symbol',
                    'CategoryID', 'Short_Question_Text']

columns_to_keep = ['StateDesc', 'Category', 'CityName', 'UniqueID', 'GeographicLevel', 'DataValueTypeID',
                   'PopulationCount', 'CityFIPS', 'TractFIPS', 'GeoLocation']

random_state = 10

In [25]:
def get_data(path):
    data = pd.read_csv(path)

    # drop unecessary columns
    data = data.drop(columns=columns_to_drop)
    # print(data.columns)

    # remove age-adjusted data
    data = data.drop(data[data.DataValueTypeID == 'AgeAdjPrv'].index)

    census_tract_data = data[data['GeographicLevel'] == 'Census Tract']
    city_data = data[data['GeographicLevel'] == 'City']

    tract_pv = census_tract_data.pivot_table(index=['StateDesc', 'CityName', 'UniqueID'], columns='MeasureId',
                                             values='Data_Value',
                                             aggfunc='sum')
    print("Size of census tract data:", len(tract_pv))  # 28004
    tract_pv = tract_pv.fillna(tract_pv.mean())

    city_pv = city_data.pivot_table(index=['StateDesc', 'CityName', 'UniqueID'], columns='MeasureId',
                                    values='Data_Value', aggfunc='sum')
    print("Size of city data:", len(city_pv))  # 500
    city_pv = city_pv.fillna(city_pv.mean())

    city_pv.reset_index(level=0, inplace=True)
    tract_pv.reset_index(level=0, inplace=True)

    return data, city_pv, tract_pv

In [26]:
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [38]:
def multi_model_analysis(clean_data):
    # split data to training and testing set
    x_data = clean_data.loc[:, clean_data.columns.isin(prevention_cols + behavior_cols)]
    i = 0
    for outcome_col in outcome_cols:
        print("-----------------------------------")
        print("#", i)

        print("Health Outcome to analyze: ", outcome_col)
        i += 1

        y_data = clean_data.loc[:, outcome_col]
        # split data to training and testing set
        x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.7,
                                                            random_state=random_state)

        # Linear Regression
        reg = LinearRegression().fit(x_train, y_train)
        # print(reg.intercept_)
        print("lm coef:", reg.coef_)
        print("lm train score:", round(reg.score(x_train, y_train), 4))
        print("lm test score:", round(reg.score(x_test, y_test), 4))

        #     X = sm.add_constant(x_train)
        #     lm = sm.OLS(y_train, X).fit()
        #     print(lm.summary())

        # Ridge Regression
        reg = Ridge().fit(x_train, y_train)
        print(reg.intercept_)
        print(reg.coef_)
        print(reg)
        print("Ridge train score:", reg.score(x_train, y_train))
        print("Ridge test score:", reg.score(x_test, y_test))

        # Lasso Regression
        reg = Lasso().fit(x_train, y_train)
        print("Lasso coef:", reg.coef_)
        print("Lasso train score:", round(reg.score(x_train, y_train), 4))
        print("Lasso test score:", round(reg.score(x_test, y_test), 4))

        # SVM - regression with Hyper-parameter Tuning
        svr = SVR()
        parameters = {'kernel': ('linear', 'rbf'), 'C': [0.01, 1, 10]}
        clf = GridSearchCV(svr, parameters, cv=10).fit(x_train, y_train)
        best_param = clf.best_params_
        print("svm best params:", clf.best_params_)
        print("svm best score:", clf.best_score_)
        print("svm best set train accuracy:", round(r2_score(y_train, clf.predict(x_train)), 4))
        print("svm best set test accuracy:", round(r2_score(y_test, clf.predict(x_test)), 4))
        # feature selection - recursive feature elimination
        estimator = SVR(kernel=best_param['kernel'], C=best_param['C'])
        selector = RFE(estimator, 5, step=1)  # top 5 features
        selector = selector.fit(x_train, y_train)
        # print(x_test.columns)
        # print(selector.support_)
        # print(selector.ranking_)
        print("top features:", x_test.columns[selector.support_])


# Top n Feature selection on given data for the specified columns
def feature_selection(clean_data, cols, n=5):
    train_score = []
    test_score = []
    top_feature = []
    for outcome_col in cols:
        x_data = clean_data.loc[:, clean_data.columns.isin(prevention_cols + behavior_cols)]
        y_data = clean_data.loc[:, outcome_col]

        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x_data)

        x_train_std, x_test_std, y_train, y_test = train_test_split(x_scaled, y_data, train_size=0.7,
                                                                    random_state=random_state)

        param_grid = {'kernel': ['linear'], 'C': [0.01, 1, 10]}
        clf = GridSearchCV(SVR(), param_grid, cv=5)
        clf = clf.fit(x_train_std, y_train)

        train_scr = r2_score(y_train, clf.predict(x_train_std))
        test_scr = r2_score(y_test, clf.predict(x_test_std))

        train_score.append(train_scr)
        test_score.append(test_scr)

        selector = RFE(clf.best_estimator_, 1, step=1)
        selector = selector.fit(x_train_std, y_train)
        top_feature.append(sorted(zip(selector.ranking_, x_data.columns.tolist()), key=lambda x: x[0])[:n])
    d = {'health_outcomes': cols, 'train_score': train_score, 'test_score': test_score, 'top_feature': top_feature}
    return pd.DataFrame(data=d)


def visual_data_prep(clean_data, prev_data, cols, n=5):
    df = pd.DataFrame()
    for outcome_col in cols:
        print("analysing ", outcome_col)
        x_data = clean_data.loc[:, clean_data.columns.isin(prevention_cols + behavior_cols)]
        y_data = clean_data.loc[:, outcome_col]

        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x_data)

        x_train_std, x_test_std, y_train, y_test = train_test_split(x_scaled, y_data, train_size=0.7,
                                                                    random_state=random_state)

        param_grid = {'kernel': ['linear'], 'C': [0.01, 1, 10]}
        clf = GridSearchCV(SVR(), param_grid, cv=5)
        clf = clf.fit(x_train_std, y_train)

        train_scr = r2_score(y_train, clf.predict(x_train_std))
        test_scr = r2_score(y_test, clf.predict(x_test_std))

        selector = RFE(clf.best_estimator_, 1, step=1)
        selector = selector.fit(x_train_std, y_train)
        arr_tup = sorted(zip(selector.ranking_, x_data.columns.tolist()), key=lambda x: x[0])[:n]

        prevention = [x[1] for x in arr_tup]
        prevention_rank = [x[0] for x in arr_tup]
        d = {'Prevention': prevention,'PreventionRank':prevention_rank}
        temp = pd.DataFrame(data=d)

        temp['Outcome'] = outcome_col
        temp['PredictionAccuracyScore'] = train_scr

        row = prev_data[prev_data.MeasureId == outcome_col]

        temp['OutcomePrevalence'] = row.Data_Value.iloc[0]
        temp['PreventionPrevalence'] = [prev_data[prev_data.MeasureId == p].Data_Value.iloc[0] for p in prevention]
        df = pd.concat([temp, df])

    return df


def append_existing_data(df, prev_data):
    temp = pd.DataFrame()
    for outcome_col in outcome_cols:
        outcome_row = prev_data[prev_data.MeasureId == outcome_col]

        df2 = df[df['Outcome'] == outcome_col]
        # Weighted average prevalence for the outcome
        df2['OutcomePrevalence'] = np.sum(outcome_row.Data_Value * outcome_row.PopulationCount) / np.sum(
            outcome_row.PopulationCount)
        prevention = df2['Prevention'].iloc[:]

        p_list = []
        for p in prevention:
            prevention_row = prev_data[prev_data.MeasureId == p]
            # Weighted average prevalence for the prevention
            prevention_prevalence = np.sum(
                prevention_row.Data_Value * prevention_row.PopulationCount) / np.sum(
                prevention_row.PopulationCount)
            p_list.append(prevention_prevalence)

        df2['PreventionPrevalence'] = p_list
        temp = pd.concat([temp, df2])
    return temp

In [39]:
import logging
import sys
import warnings

In [ ]:
def initial_setup():
    warnings.filterwarnings("ignore")  # ignore warnings from sklean
    global logger
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    logger.addHandler(ch)
    logger.info("Logger ready")


def main():
    initial_setup()
    print("Accepted mode: \n1. Perform top feature selection Analysis.\n"
          "2. Generate output file for visualization.\n"
          "3. Run multiple models comparision analysis.\n")
    try:
        mode = int(input('Enter the number of your selected mode:\n'))
    except ValueError:
        print("Not a number")

    # Local path to store the raw data
    path = "/content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv"
    data, city_pv, tract_pv = get_data(path)
    tract_pv.reset_index(level=0, inplace=True)
    logging.info("Get formatted pivot table data by city and census track from file: " + path)

    if mode == 1:
        print("You have chose to run top feature selection analysis for health care data")
        top_n = int(input('Enter the number of top features you want to select:\n'))

        logging.info("About to run feature selection for city level data")
        feature_selection_df = feature_selection(city_pv, outcome_cols, top_n)
        feature_selection_df.to_csv("city_data.csv")
        logging.info("Country data has been saved to csv")

        logging.info("About to run feature selection for each state")
        states = np.unique(tract_pv['StateDesc'])
        state_feature_selection_df = pd.DataFrame()
        for state in states:
            logging.info("Feature selection for state %s" % state)
            state_data = tract_pv[tract_pv['StateDesc'] == state]
            df = feature_selection(state_data, outcome_cols, top_n)
            df["state"] = state
            state_feature_selection_df = pd.concat([state_feature_selection_df, df])
        state_feature_selection_df.to_csv("state_feature_selection.csv")
        logging.info("States data has been saved to csv")
    elif mode == 2:
        print("You have chosen to generate output file for visualization")

        result = pd.DataFrame()
        logging.info("Start data prep in US level")
        prev_data = data[data['GeographicLevel'] == 'US']

        #df = visual_data_prep(city_pv, prev_data, outcome_cols, 5)
        df = visual_data_prep(tract_pv, prev_data, outcome_cols, 5)
        df['Region'] = 'US'
        df['Level'] = 'US'
        df['PreventionCalculationLevel'] = 'National'
        result = pd.concat([result, df])
        logging.info("Complete data prep in US level")

        logging.info("Start data prep in State level")
        states = np.unique(tract_pv['StateDesc'])
        state_df = pd.DataFrame()
        for state in states:
            state_data = tract_pv[tract_pv['StateDesc'] == state]
            logging.info("Data prep for state %s with %s census tracks" % (state, len(state_data)))
            prev_data = data[data['GeographicLevel'] == 'City']
            prev_data = prev_data[prev_data['StateDesc'] == state]

            # for state has more than 100 data points, we conduct another analysis on state level
            if len(state_data) > 100:
                df = visual_data_prep(state_data, prev_data, outcome_cols, 5)
                df['PreventionCalculationLevel'] = 'State'
                df["Region"] = state
                df['Level'] = 'State'
                state_df = pd.concat([state_df, df])

            # Append national level data for each state with state prevalence information
            df = append_existing_data(result[result.Level == 'US'], prev_data)
            df["Region"] = state
            df['Level'] = 'State'
            df['PreventionCalculationLevel'] = 'National'
            state_df = pd.concat([state_df, df])

            # For each cities with in the state
            cities = np.unique(state_data['CityName'])
            national_calc_df = state_df[state_df.PreventionCalculationLevel == 'National']
            national_calc_df = national_calc_df[national_calc_df.Region == state]
            state_calc_df = state_df[state_df.PreventionCalculationLevel == 'State']
            state_calc_df = state_calc_df[state_calc_df.Region == state]

            city_df = pd.DataFrame()
            for city in cities:
                # Apply national level analysis result for each city
                city_prev_data = prev_data[prev_data.CityName == city]
                df = append_existing_data(national_calc_df, city_prev_data)
                df["Region"] = city
                df['Level'] = 'City'
                city_df = pd.concat([city_df, df])

                # Apply state level analysis result for each city if exists
                if state_calc_df.shape[0] > 0:
                    df = append_existing_data(state_calc_df, city_prev_data)
                    df["Region"] = city
                    df['Level'] = 'City'
                    city_df = pd.concat([city_df, df])
            result = pd.concat([result, city_df])
        result = pd.concat([result, state_df])
        col_order = ['Region','Level','Outcome','Prevention','OutcomePrevalence','PredictionAccuracyScore','PreventionPrevalence','PreventionRank','PreventionCalculationLevel']
        result = result [col_order]
        logging.info("Complete data prep in State level")

        result.to_csv("visual_input_tract_with_city.csv", index=False)
        logging.info("csv file for visualization has been generated")
    elif mode == 3:
        print("You have choose to run multiple models comparision analysis.\n")
        level = input("Enter the level of analysis you want to perform (State/US):\n")
        if level == 'US':
            print("About to analysis on national level")
            multi_model_analysis(city_pv)
        elif level == 'State':
            show_state = input("Do you wish to see all the state names? (Y/N)")
            if show_state == 'Y':
                print( np.unique(tract_pv['StateDesc']) )

            state = input("Enter the name of the state you want to analyze:\n")
            state_data = tract_pv[tract_pv['StateDesc'] == state]
            if state_data.shape[0] == 0:
                print("Bad state name, abort!")
                sys.exit()
            print("About to run model selection for state: ", state, " with ", len(state_data), " census track.")
            multi_model_analysis(state_data)
        else:
            print("Unacceptable input, abort!")


main()

INFO:root:Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready
Logger ready


Accepted mode: 
1. Perform top feature selection Analysis.
2. Generate output file for visualization.
3. Run multiple models comparision analysis.

Enter the number of your selected mode:
3


INFO:root:Get formatted pivot table data by city and census track from file: /content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv
Get formatted pivot table data by city and census track from file: /content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv
Get formatted pivot table data by city and census track from file: /content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv
Get formatted pivot table data by city and census track from file: /content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv
Get formatted pivot table data by city and census track from file: /content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv
Get formatted pivot table data by city and census track from file: /content/gdrive/MyDrive/500_Cities__Local_Data_for_Better_Health__2018_release (1).csv
Get formatted pivot table data by city and census track from file:

Size of census tract data: 28004
Size of city data: 500
You have choose to run multiple models comparision analysis.

Enter the level of analysis you want to perform (State/US):
US
About to analysis on national level
-----------------------------------
# 0
Health Outcome to analyze:  ARTHRITIS
lm coef: [-0.04951063 -0.29566126  0.49716485 -0.17651153  0.18102031  0.12261768
 -0.00839581 -0.11227574  0.62315696  0.09225053 -0.03787054 -0.17382469
  0.15760971 -0.01171096  0.02264826]
lm train score: 0.8455
lm test score: 0.8268
-19.662916069664067
[-0.04978606 -0.29530243  0.49697395 -0.17619328  0.18093421  0.12258824
 -0.00839234 -0.11218978  0.62245362  0.09202337 -0.03763863 -0.17398862
  0.15778882 -0.01171708  0.0225762 ]
Ridge()
Ridge train score: 0.8455311571160908
Ridge test score: 0.8268958877709699
Lasso coef: [-0.12130705 -0.0303183   0.466717   -0.          0.05109257  0.
 -0.         -0.          0.34877278  0.          0.         -0.10402164
  0.14695428 -0.01634848  0.  